# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 122MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Wed May 19 12:41:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB 

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

  Using cached https://files.pythonhosted.org/packages/81/91/61d69d58a1af1bd81d9ca9d62c90a6de3ab80d77f27c5df65d9a2c1f5626/transformers-4.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/04/5b870f26a858552025a62f1649c20d29d2672c02ff3c3fb4c688ca46467a/tokenizers-0.10.2-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/75/ee/67241dc87f266093c533a2d4d3d69438e57d7a90abb216fa076e7d475d4a/sacremoses-0.0.45-py3-none-any.whl


## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [4]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 2.9MB/s 


## Load Model and Tokenizer




 

In [5]:
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [6]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [7]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [8]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 450
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 350

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2 + random.randint(-100, 100), len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [9]:
def evaluate(data, output, cnt, mode, doc_stride):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            if '[UNK]' in answer:
                start_index -= len(test_questions[cnt]["question_text"])+2
                end_index -= len(test_questions[cnt]["question_text"])+2
                if mode == "dev":
                    answer = dev_paragraphs[dev_questions[cnt]["paragraph_id"]][k*doc_stride + start_index : k*doc_stride + end_index + 1]
                else:
                    try:
                        start_char_idx = test_paragraphs_tokenized[test_questions[cnt]["paragraph_id"]].token_to_chars(k * doc_stride + start_index)[0]
                        end_char_idx = test_paragraphs_tokenized[test_questions[cnt]["paragraph_id"]].token_to_chars(k * doc_stride + end_index)[0]
                        answer = test_paragraphs[test_questions[cnt]["paragraph_id"]][start_char_idx : end_char_idx + 1]
                    except:
                        answer = test_paragraphs[test_questions[cnt]["paragraph_id"]][k*doc_stride + start_index : k*doc_stride + end_index + 1]


    
    #print(answer)
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [10]:
num_epoch = 2 
validation = True
logging_step = 100
learning_rate = 3e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        step += 1

        ##### TODO: Apply linear learning rate decay #####
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0
        scheduler.step()

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, i, "dev", doc_stride) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


Epoch 1 | Step 100 | loss = 2.517, acc = 0.325
Epoch 1 | Step 200 | loss = 0.985, acc = 0.605
Epoch 1 | Step 300 | loss = 0.987, acc = 0.627
Epoch 1 | Step 400 | loss = 0.837, acc = 0.675
Epoch 1 | Step 500 | loss = 0.838, acc = 0.700
Epoch 1 | Step 600 | loss = 0.814, acc = 0.665
Epoch 1 | Step 700 | loss = 0.790, acc = 0.702
Epoch 1 | Step 800 | loss = 0.793, acc = 0.688
Epoch 1 | Step 900 | loss = 0.879, acc = 0.675
Epoch 1 | Step 1000 | loss = 0.759, acc = 0.707
Epoch 1 | Step 1100 | loss = 0.756, acc = 0.700
Epoch 1 | Step 1200 | loss = 0.688, acc = 0.710
Epoch 1 | Step 1300 | loss = 0.721, acc = 0.710
Epoch 1 | Step 1400 | loss = 0.579, acc = 0.770
Epoch 1 | Step 1500 | loss = 0.689, acc = 0.715
Epoch 1 | Step 1600 | loss = 0.660, acc = 0.725
Epoch 1 | Step 1700 | loss = 0.766, acc = 0.705
Epoch 1 | Step 1800 | loss = 0.717, acc = 0.720
Epoch 1 | Step 1900 | loss = 0.629, acc = 0.743
Epoch 1 | Step 2000 | loss = 0.599, acc = 0.757
Epoch 1 | Step 2100 | loss = 0.640, acc = 0.767
E

杜 拜 知 識 村
3500 萬 美 元
《 胭 脂 扣 》
臺 東 市
顏 延 之
1985 年
長 江 流 域
1920 年
賓 夕 法 尼 亞 州
賓 夕 法 尼 亞 州
商 稅
校 園 核 能 研 究 反 應 爐
私 立 研 究 型 大 學
《 大 明 會 典
2017 年
勸 誡 性 長 詩
1951 年
台 灣 海 峽
光 緒 31 年
經 濟 力
9 日
紐 澤 西 美 洲 人 」
昭 和 四 十 九 年
1922 年
1793 年
英 國
元 朝
西 夏
韓 國 電 影
王 莽
心 理 危 機 干 預 熱 線
冉 閔
朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
草 書
1984 年
內 蒙 古
紅 土
新 教 徒 對 聖 經 正 典 的 質 疑
不 確 定 性
新 加 坡
阿 伊 努 人
蘇 門 答 臘 島
加 拿 大
一 萬 年
丹 麥
索 馬 利 亞
位 於 天 樂 里
消 費 者
機 器 人 系 列
查 理 斯 · 萊 爾
九 龍 東
九 龍 塘 省 善 真 堂
一 個 月
傍 晚 時 份
1909 年
《 德 意 志 民 主 共 和 國 憲 法 》
肝 臟
頭 目 職 位
科 尼 亞
俄 語
《 大 公 報 》
網 球
波 蘭
白 米
比 例 代 表 制
降 雨 的 不 均 衡 和 英 國 人 的 經 濟 管 理 政 策 失 誤
公 共 衛 生 問 題
闢 為 公 園 ， 正 式 對 民 眾 開 放
《 周 禮 · 考 工 記 》
1971 年 4 月
2015 年
阿 拉 巴 馬 州
董 必 武
正 式 承 認 放 棄 瓦 罕 帕 米 爾
義 大 利
日 軍
1917 年
商 朝 時 期
支 持
景 宗
屋 大 維
《 法 蘭 克 福 條 約 》
毛 澤 東
波 蘭
中 宣 部
2012 年
四 面 環 海
1928 年
瑞 士 近 衛 隊
中 國 鐵 道 部
新 北 市
約 翰
澳 門
1611 年
狄 托
法 西 斯 獨 裁 政 權
牛 牧 寺
光 球 層
毛 澤 東
2000 年
蘇 維 埃 俄 國
元 祿 年 間
害 怕 被 人 聽 到 自 己 說 上 海 話 會 被 認 為 大 陸 人
1920 年
汐 止 區
劉 培 基
孟 加 拉 國
近 衛
3 月 8 日
《 國 際 動 物 命 名 規 約 》
1987 年
[

Epoch 2 | Step 100 | loss = 0.208, acc = 0.880
Epoch 2 | Step 200 | loss = 0.283, acc = 0.885
Epoch 2 | Step 300 | loss = 0.292, acc = 0.860
Epoch 2 | Step 400 | loss = 0.223, acc = 0.880
Epoch 2 | Step 500 | loss = 0.219, acc = 0.900
Epoch 2 | Step 600 | loss = 0.305, acc = 0.868
Epoch 2 | Step 700 | loss = 0.239, acc = 0.870
Epoch 2 | Step 800 | loss = 0.283, acc = 0.880
Epoch 2 | Step 900 | loss = 0.218, acc = 0.887
Epoch 2 | Step 1000 | loss = 0.276, acc = 0.857
Epoch 2 | Step 1100 | loss = 0.342, acc = 0.837
Epoch 2 | Step 1200 | loss = 0.219, acc = 0.900
Epoch 2 | Step 1300 | loss = 0.311, acc = 0.865
Epoch 2 | Step 1400 | loss = 0.317, acc = 0.850
Epoch 2 | Step 1500 | loss = 0.225, acc = 0.868
Epoch 2 | Step 1600 | loss = 0.292, acc = 0.897
Epoch 2 | Step 1700 | loss = 0.327, acc = 0.847
Epoch 2 | Step 1800 | loss = 0.189, acc = 0.905
Epoch 2 | Step 1900 | loss = 0.322, acc = 0.847
Epoch 2 | Step 2000 | loss = 0.193, acc = 0.910
Epoch 2 | Step 2100 | loss = 0.243, acc = 0.862
E

杜 拜 知 識 村
3500 萬 美 元
《 胭 脂 扣 》
臺 東 市
顏 延 之
1985 年
長 江 流 域
1920 年
賓 夕 法 尼 亞 州
賓 夕 法 尼 亞 州
商 稅
校 園 核 能 研 究 反 應 爐
私 立 研 究 型 大 學
《 涌 幢 小 品 · 總 督 總 兵 》
2017 年
勸 誡 性 長 詩
1951 年
台 灣 海 峽
光 緒 31 年
經 濟 力
9 日
紐 澤 西 美 洲 人
昭 和 四 十 九 年
1922 年
1793 年
英 國
元 朝
西 夏
電 影
王 莽
心 理 危 機 干 預 熱 線
冉 閔
怕 他 與 燕 王 呵 成 一 氣
草 書
1984 年
內 蒙 古
紅 土
鑑 於 新 教 徒 對 聖 經 正 典 的 質 疑
不 確 定 性
新 加 坡
「 蝦 夷 地
蘇 門 答 臘 島
加 拿 大
公 元 前 一 萬 年
丹 麥 人
索 馬 利 亞
天 樂 里
消 費 者
機 器 人 系 列
查 理 斯 · 萊 爾
九 龍 東
九 龍 塘 省 善 真 堂
一 個 月
傍 晚 時 份
1909 年
《 德 意 志 民 主 共 和 國 憲 法 》 第 1 章 ， 第 1 條
肝 臟
英 制
科 尼 亞
俄 語
《 大 公 報 》
網 球 比 賽
波 蘭
白 米
比 例 代 表 制
管 理 政 策 失 誤
公 共 衛 生 問 題
闢 為 公 園
《 周 禮 · 考 工 記 》
1971 年 4 月
2015 年
阿 拉 巴 馬 州
董 必 武
正 式 承 認 放 棄 瓦 罕 帕 米 爾
義 大 利
日 軍
1917 年
商 朝 時 期
支 持
景 宗
屋 大 維
《 法 蘭 克 福 條 約 》
毛 澤 東
波 蘭
中 宣 部
2012 年
四 面 環 海
1928 年
瑞 士 近 衛 隊
中 國 鐵 道 部
新 北 市
約 翰
澳 門
1611 年
狄 托
法 西 斯
牛 牧 寺
光 球 層
毛 澤 東
2000 年
蘇 維 埃 俄 國
元 祿 年 間
害 怕 被 人 聽 到 自 己 說 上 海 話 會 被 認 為 大 陸 人
1920 年
汐 止 區
梅 艷 芳
孟 加 拉 國
近 衛 頭 銜
3 月
《 國 際 動 物 命 名 規 約 》
1987 年
以 為 高 句 麗 滅 亡 了
1912 年
金 剛 計 劃
突 

## Testing

In [11]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


白 色 羽 毛
《 諸 羅 縣 志 》
袁 晁
柏 林 陸 軍 學 院
決 勝 局
聶 耳
1884 年
西 漢 的 版 圖 向 西 擴 張
英 國
醜 化 對 手 的 作 用
周 星 馳
三 中 老
無 障 礙 環 境
anticipation
2004 年
硫 化 氫
華 萊 士
外 面 的 部 份 溫 度 會 比 內 部 核 心 的 低
經 濟
100 公 里
愛 爾 蘭 語
荷 蘭
硫 磺
塞 爾 維 亞 反 對 黨
1910 年 代
外 交 活 動
拉 丁 文
猶 太 基 督 徒
1964 年
霍 克 菲 爾 德
被 迫 外 遷
《 薩 蒂 里 卡 》
灣 仔
爪 哇
一 種 無 限 制 的 、 自 私 、 而 野 蠻 的 競 爭 情 況
1968 年
新 北 市
小 白 宮
沒 有 任 何 示 威 申 請 得 到 當 局 通 過
南 人
宋 端 宗
四 千 名
46 億 年
1070 年
國 民 養 老 保 險 法
水 上 報 告 廳
1929 年
航 天 技 術
1932 年
大 型 購 物 中 心 megabox 開 幕
康 熙 帝
自 強 學 堂
1960 年 代
亞 歷 山 大 一 世 被 刺 殺
紐 約 市
基 督 教
預 覽 阿 爾 法 版 本
100 萬 以 上
新 北 市
佔 領 行 動
路 易 十 五
中 國 國 民 黨
中 生 代 侏 羅 紀
阿 拉 貢 國 王
熱 帶 雨 林 氣 候
意 圖 完 全 擺 脫 漢 朝 的 控 制
蔣 中 正
東 漢
前 三 點 起 落 架
弘 治 十 二 年
亞 洲
1996 年
德 意 志 聯 邦 共 和 國
氦
西 班 牙 人 入 侵 納 瓦 爾
交 通 運 輸 部
高 麗 大 學 校 主 樓
朱 少 文
羽 柴 秀 吉
俾 斯 麥
金 泳 三
1958 年
靈 魂 」
科 龍 港
蓋 爾 人
四 次 重 差 觀 測 術
西 漢 元 封 二 年
隋 滅 南 陳
佛 教
1861 年
豐 臣 秀 吉
三 吳 地 區
英 國
大 躍 進 運 動
國 立 武 昌 高 等 師 範 學 校
蘇 利 南
1934 年
檸 檬 酸 咖 啡 因 療 法
1913 年
1949 年
大 象
1915 年
1000 年
有 絲 分 裂
德 國
高 盧 帝 國
仁 祖
十 個
佛 學 論 書
臺 中 高 農
歐 登 堡